# Q12

In [3]:
import pandas as pd
import os, pickle
import string
import re
import numpy as np
from tqdm import tqdm as timer
from sklearn import linear_model
import sklearn.metrics as mcs
from collections import defaultdict
import time

In [4]:
fname = "movie_genre.txt"

start = time.time()
movie_genre_dict = {}
line_count = 0

with open(fname, encoding="ISO-8859-1") as fin:
    for line in fin:
        line_count += 1
        obj_list = re.split(r"\t+", line) ############# change to \t        

        obj = obj_list[0]

        p1 = re.search(r"\((\d{4}|\?{4})[^()]*\)(.+)", obj)
        if p1:
            obj = obj.replace(p1.group(2),"")
            movie_genre_dict[obj]=obj_list[1]
        else:
            p2 = re.search(r"\((\d{4}|\?{4})[^()]*\)", obj)
            if not p2:
                obj = obj + " (????)"
            movie_genre_dict[obj]=obj_list[1]


with open("movie_genre_clean.txt", "w", encoding="utf-8") as outfile:
    for k, v in movie_genre_dict.items():
        outfile.write("%s\t%s" %(k, v)) 

TypeError: 'encoding' is an invalid keyword argument for this function

In [2]:
def clean_string(s):
    return(re.sub(r'\(.*\)|\{.*\}|\'|\"', "", s).lstrip().rstrip())

In [3]:
m2r = {}
printable = set(string.printable)

with open ('./movie_rating.txt') as infile:
    for line in timer(infile, total=348547, desc='movie_rating.txt'):
        line = filter(lambda x : x in printable, line.strip().translate(None, "&$ "))
        arr = line.split('\t\t')
        m2r[clean_string(arr[0])] = arr[1]
        
pickle.dump(m2r, open("m2r_.pkl", "wb" ))

M2R = pd.DataFrame(m2r.items(), columns=['movie', 'rating'])
M2R.to_pickle("M2R.pkl")

movie_rating.txt: 100%|██████████| 348547/348547 [00:05<00:00, 66843.22it/s]


In [4]:
m2d = {}
printable = set(string.printable)

with open ('./director_movies.txt') as infile:
    for line in timer(infile, total=348547, desc='director_movies.txt'):
        line = filter(lambda x : x in printable, line.strip().translate(None, "&$ "))
        arr = line.split('\t\t')
        for movie in arr[1:]:
            m2d[clean_string(movie)] = arr[0]

pickle.dump(m2d, open("m2d_.pkl", "wb" ))
                      
M2D = pd.DataFrame(m2d.items(), columns=['movie', 'director'])
M2D.to_pickle("M2D.pkl")

director_movies.txt: 389663it [00:11, 33662.73it/s]                            


In [5]:
M2R2D = pd.merge(M2R, M2D, how='inner')

In [6]:
#print M2R2D

In [7]:
M2R2D[['rating']] = M2R2D[['rating']].apply(pd.to_numeric)

In [8]:
DR = M2R2D.groupby(['director'])
D2R = pd.DataFrame(index=range(0,len(M2R2D.director.unique())), columns=['director', 'rating'])
    
count = 0
for name, group in timer(DR):
    D2R.loc[[count],'director'] = name
    D2R.loc[[count],'rating'] = group['rating'].mean()
    count += 1

100%|██████████| 98766/98766 [42:28<00:00, 38.76it/s]


In [9]:
D2R.to_pickle("D2R.pkl")

In [10]:
d2r = list(D2R.set_index('director').to_dict().values()).pop()
pickle.dump(d2r, open("d2r_.pkl", "wb" ))

In [11]:
d2r["zuyar,Ali"]

6.1

### Actor/Actress clean

In [12]:
infiles = ["./actor_movies.txt", "./actress_movies.txt"]
printable = set(string.printable)
fl = {"./actor_movies.txt": 2167653, "./actress_movies.txt": 1182813}

a2m = defaultdict(set)
m2a = defaultdict(set)

for f in infiles:
    with open(f, "rb") as infile:
        for line in timer(infile, total=fl[f], desc=f.split('/')[1]):
            if line.count("\t\t") <= 10:
                continue
            line = filter(lambda x : x in printable, line.strip().translate(None, "&$ "))
            arr = line.split("\t\t")
            
            actor = arr[0]
            movies = map(clean_string, arr[1:])
            
            for m in movies:
                a2m[actor].add(m)
                m2a[m].add(actor)
                
large_movies = set(filter(lambda x : len(m2a[x]) > 10, m2a.keys()))
a2m = {actor : movies.intersection(large_movies) for (actor, movies) in a2m.iteritems()}
a2m = {actor : movies for (actor, movies) in a2m.iteritems() if len(movies) > 0}
m2a = {movie : actors for (movie, actors) in m2a.iteritems() if len(actors) > 10}

pickle.dump(a2m, open("a2m_.pkl", "wb" ))
pickle.dump(m2a, open("m2a_.pkl", "wb" ))

actress_movies.txt: 100%|██████████| 1182813/1182813 [00:14<00:00, 82333.64it/s]


In [13]:
A2M = pd.DataFrame(a2m.items(), columns=['actor', 'movie'])
M2A = pd.DataFrame(m2a.items(), columns=['movie', 'actor'])
A2M.to_pickle("A2M.pkl")
M2A.to_pickle("M2A.pkl")

### Actor pagerank

In [14]:
from igraph import *

In [15]:
# create actor edge list
count = 0
outfile = open("actor_edgelist.txt", "w")

for i, movies in timer(a2m.iteritems(), total=len(a2m)):
    # for an actor i, get the list of actors that appear in the same movie
    i2k = defaultdict(int)
    for j in movies:
        for k in m2a[j]:
            i2k[k]=i2k[k]+1
    for k, w in i2k.iteritems():
        weight = float(w) / len(movies)
        line = i + '\t' + k + '\t' + str(weight) + '\n'
        outfile.write(line)
        count = count + 1
outfile.close()
print(count)

100%|██████████| 99652/99652 [02:57<00:00, 562.28it/s]

42135556


In [16]:
g = Graph.Read_Ncol('actor_edgelist.txt', directed=True)

In [17]:
page_rank = g.pagerank(vertices=None, directed=True)
a2pr = {}
for idx, v in enumerate(g.vs):
    a2pr[v["name"]] = page_rank[idx]

In [18]:
pickle.dump(a2pr, open("a2pr_.pkl", "wb" ))
A2PR = pd.DataFrame(a2pr.items(), columns=['actor', 'pagerank'])
A2PR.to_pickle("A2PR.pkl")

### Build features

In [28]:
A2PR = pd.read_pickle("A2PR.pkl")
A2M = pd.read_pickle("A2M.pkl")
M2A = pd.read_pickle("M2A.pkl")
M2D = pd.read_pickle("M2D.pkl")
M2R = pd.read_pickle("M2R.pkl")
D2R = pd.read_pickle("D2R.pkl")

M2R = M2R.drop([0])

movie_with_AR = pd.merge(M2A, M2R, how='inner')
movie_with_ARD = pd.merge(movie_with_AR, M2D, how='inner')
avail_movies = movie_with_ARD['movie'].values
pickle.dump(avail_movies, open("avail_movies.pkl", "wb"))

In [32]:
a2m = pickle.load(open("a2m_.pkl", "rb" ))
m2a = pickle.load(open("m2a_.pkl", "rb" ))
a2pr = pickle.load(open("a2pr_.pkl", "rb" ))
d2r = pickle.load(open("d2r_.pkl", "rb" ))
m2d = pickle.load(open("m2d_.pkl", "rb" ))
avail_movies = pickle.load(open("avail_movies.pkl", "rb"))

In [40]:
def build_feature(movie):
    features = [0] * 7
    actors = m2a[movie]
    page_ranks = []
    for actor in actors:
        page_ranks.append(a2pr[actor])
    sorted_page_ranks = sorted(page_ranks, reverse=True)
    features[:6] = sorted_page_ranks[:6]
    
    features[6] = d2r[m2d[movie]]

    return np.array(features)

feature_vector = np.vstack(map(build_feature, timer(avail_movies)))

100%|██████████| 74678/74678 [00:02<00:00, 30085.77it/s]


In [41]:
actual_rating = []
for movie in timer(avail_movies):
    actual_rating.append(float(m2r[movie]))
rating_vector = np.array(actual_rating)

100%|██████████| 74678/74678 [00:00<00:00, 412580.76it/s]


### Linear Regression model training

In [42]:
model = linear_model.LinearRegression()
model.fit(feature_vector, rating_vector)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [43]:
test_movies = [
    "Batman v Superman: Dawn of Justice (2016)",
    "Mission: Impossible - Rogue Nation (2015)",
    "Minions (2015)"
]
clean_test_movies = []
for movie in test_movies:
    clean_test_movies.append(clean_string(movie.translate(None, "&$ ")))

In [44]:
test_feature_vector = map(build_feature, clean_test_movies)
model.predict(test_feature_vector)

array([7.01318827, 6.80856276, 6.69704493])

### RMSE

In [45]:
prediction = model.predict(feature_vector)

In [46]:
mcs.mean_squared_error(rating_vector, prediction)

0.7090275567049534